We download the meteo data using openmeteo api, we use many parameters to improve our model

In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 50.768745,
	"longitude": 16.269667,
	"start_date": "2015-01-01",
	"end_date": "2024-01-01",
	"daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "precipitation_sum", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "cape_mean", "cloud_cover_mean", "cloud_cover_max", "cloud_cover_min", "dew_point_2m_mean", "surface_pressure_mean", "surface_pressure_max", "surface_pressure_min", "relative_humidity_2m_mean", "relative_humidity_2m_max", "relative_humidity_2m_min", "wind_speed_10m_mean", "wind_speed_10m_min", "cape_max", "cape_min", "precipitation_hours"],
	"hourly": "boundary_layer_height",
	"wind_speed_unit": "ms",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_boundary_layer_height = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["boundary_layer_height"] = hourly_boundary_layer_height

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(5).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(6).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(7).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(8).ValuesAsNumpy()
daily_cape_mean = daily.Variables(9).ValuesAsNumpy()
daily_cloud_cover_mean = daily.Variables(10).ValuesAsNumpy()
daily_cloud_cover_max = daily.Variables(11).ValuesAsNumpy()
daily_cloud_cover_min = daily.Variables(12).ValuesAsNumpy()
daily_dew_point_2m_mean = daily.Variables(13).ValuesAsNumpy()
daily_surface_pressure_mean = daily.Variables(14).ValuesAsNumpy()
daily_surface_pressure_max = daily.Variables(15).ValuesAsNumpy()
daily_surface_pressure_min = daily.Variables(16).ValuesAsNumpy()
daily_relative_humidity_2m_mean = daily.Variables(17).ValuesAsNumpy()
daily_relative_humidity_2m_max = daily.Variables(18).ValuesAsNumpy()
daily_relative_humidity_2m_min = daily.Variables(19).ValuesAsNumpy()
daily_wind_speed_10m_mean = daily.Variables(20).ValuesAsNumpy()
daily_wind_speed_10m_min = daily.Variables(21).ValuesAsNumpy()
daily_cape_max = daily.Variables(22).ValuesAsNumpy()
daily_cape_min = daily.Variables(23).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(24).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["cape_mean"] = daily_cape_mean
daily_data["cloud_cover_mean"] = daily_cloud_cover_mean
daily_data["cloud_cover_max"] = daily_cloud_cover_max
daily_data["cloud_cover_min"] = daily_cloud_cover_min
daily_data["dew_point_2m_mean"] = daily_dew_point_2m_mean
daily_data["surface_pressure_mean"] = daily_surface_pressure_mean
daily_data["surface_pressure_max"] = daily_surface_pressure_max
daily_data["surface_pressure_min"] = daily_surface_pressure_min
daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
daily_data["relative_humidity_2m_max"] = daily_relative_humidity_2m_max
daily_data["relative_humidity_2m_min"] = daily_relative_humidity_2m_min
daily_data["wind_speed_10m_mean"] = daily_wind_speed_10m_mean
daily_data["wind_speed_10m_min"] = daily_wind_speed_10m_min
daily_data["cape_max"] = daily_cape_max
daily_data["cape_min"] = daily_cape_min
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)

Coordinates: 50.79085922241211°N 16.33452033996582°E
Elevation: 441.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date  boundary_layer_height
0     2015-01-01 00:00:00+00:00                  245.0
1     2015-01-01 01:00:00+00:00                  245.0
2     2015-01-01 02:00:00+00:00                  255.0
3     2015-01-01 03:00:00+00:00                  275.0
4     2015-01-01 04:00:00+00:00                  295.0
...                         ...                    ...
78907 2024-01-01 19:00:00+00:00                    NaN
78908 2024-01-01 20:00:00+00:00                    NaN
78909 2024-01-01 21:00:00+00:00                    NaN
78910 2024-01-01 22:00:00+00:00                    NaN
78911 2024-01-01 23:00:00+00:00                    NaN

[78912 rows x 2 columns]

Daily data
                           date  temperature_2m_mean  temperature_2m_max  \
0    2015-01-01 00:00:00+00:00             1.379250            4.060500   
1    2015-01-02 00:00:00+00:

However, we must merge the hourly and daily data: bounduary layer height is very important parameter and was only available for hourly resolution

In [2]:
hourly_dataframe

,date,boundary_layer_height
0,2015-01-01 00:00:00+00:00,245.0
1,2015-01-01 01:00:00+00:00,245.0
2,2015-01-01 02:00:00+00:00,255.0
3,2015-01-01 03:00:00+00:00,275.0
4,2015-01-01 04:00:00+00:00,295.0
...,...,...
78907,2024-01-01 19:00:00+00:00,NaN
78908,2024-01-01 20:00:00+00:00,NaN
78909,2024-01-01 21:00:00+00:00,NaN
78910,2024-01-01 22:00:00+00:00,NaN


In [3]:
daily_dataframe

,date,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,...,surface_pressure_max,surface_pressure_min,relative_humidity_2m_mean,relative_humidity_2m_max,relative_humidity_2m_min,wind_speed_10m_mean,wind_speed_10m_min,cape_max,cape_min,precipitation_hours
0,2015-01-01 00:00:00+00:00,1.379250,4.060500,-0.6395,0.7,0.7,0.00,4.161730,255.283386,2.97,...,979.225342,974.452332,86.345375,97.873283,62.604813,3.429251,2.641969,NaN,NaN,5.0
1,2015-01-02 00:00:00+00:00,2.300083,4.310500,0.4105,0.6,0.6,0.00,7.783316,241.579300,1.04,...,973.224121,965.643127,84.359009,93.347153,76.609779,6.473415,4.465423,NaN,NaN,2.0
2,2015-01-03 00:00:00+00:00,0.816750,1.860500,-0.4895,0.0,0.0,0.21,6.736468,255.062943,2.81,...,970.659851,958.340332,76.418083,85.706589,67.662537,5.743515,4.477723,NaN,NaN,0.0
3,2015-01-04 00:00:00+00:00,-0.291583,0.710500,-1.6395,1.9,0.3,1.33,7.874643,309.481964,3.28,...,971.658936,958.558472,83.625832,89.970604,77.018280,6.875924,5.345091,NaN,NaN,12.0
4,2015-01-05 00:00:00+00:00,-0.847833,0.460500,-2.4895,8.4,0.0,5.88,6.881860,314.149841,0.88,...,972.343018,968.631409,93.280952,98.554314,85.192207,5.284514,3.080584,NaN,NaN,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2023-12-28 00:00:00+00:00,4.800083,6.360500,3.2605,0.0,0.0,0.00,10.430724,214.158920,3.15,...,963.549622,962.023254,76.966934,85.996208,62.241528,8.925221,7.488657,NaN,NaN,0.0
3284,2023-12-29 00:00:00+00:00,7.139667,8.410501,5.1605,0.0,0.0,0.00,9.899495,223.687241,3.39,...,961.694275,957.753113,80.429390,90.355545,72.525963,8.493058,6.862215,NaN,NaN,0.0
3285,2023-12-30 00:00:00+00:00,5.614666,7.310500,2.8605,0.1,0.1,0.00,8.504117,234.613480,3.24,...,962.934265,957.206482,80.916817,92.821678,67.846848,5.735832,3.252691,NaN,NaN,1.0
3286,2023-12-31 00:00:00+00:00,4.027167,7.310500,2.5605,0.1,0.1,0.00,8.013114,204.914337,3.69,...,962.622375,953.696045,90.388023,96.521301,79.848091,5.668894,4.275511,NaN,NaN,1.0


In [4]:
hourly_dataframe["date"] = pd.to_datetime(hourly_dataframe['date'])
hourly_dataframe = hourly_dataframe.set_index('date')
hourly_dataframe = hourly_dataframe.resample('D').mean()
hourly_dataframe

,boundary_layer_height
date,
2015-01-01 00:00:00+00:00,258.958344
2015-01-02 00:00:00+00:00,718.333313
2015-01-03 00:00:00+00:00,918.750000
2015-01-04 00:00:00+00:00,1217.916626
2015-01-05 00:00:00+00:00,765.625000
...,...
2023-12-28 00:00:00+00:00,343.750000
2023-12-29 00:00:00+00:00,836.250000
2023-12-30 00:00:00+00:00,726.666687


In [5]:
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])
daily_dataframe = daily_dataframe.set_index('date')

In [ ]:
daily_dataframe = daily_dataframe.join(hourly_dataframe)
daily_dataframe

,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,cape_mean,...,surface_pressure_min,relative_humidity_2m_mean,relative_humidity_2m_max,relative_humidity_2m_min,wind_speed_10m_mean,wind_speed_10m_min,cape_max,cape_min,precipitation_hours,boundary_layer_height
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,1.379250,4.060500,-0.6395,0.7,0.7,0.00,4.161730,255.283386,2.97,NaN,...,974.452332,86.345375,97.873283,62.604813,3.429251,2.641969,NaN,NaN,5.0,258.958344
2015-01-02 00:00:00+00:00,2.300083,4.310500,0.4105,0.6,0.6,0.00,7.783316,241.579300,1.04,NaN,...,965.643127,84.359009,93.347153,76.609779,6.473415,4.465423,NaN,NaN,2.0,718.333313
2015-01-03 00:00:00+00:00,0.816750,1.860500,-0.4895,0.0,0.0,0.21,6.736468,255.062943,2.81,NaN,...,958.340332,76.418083,85.706589,67.662537,5.743515,4.477723,NaN,NaN,0.0,918.750000
2015-01-04 00:00:00+00:00,-0.291583,0.710500,-1.6395,1.9,0.3,1.33,7.874643,309.481964,3.28,NaN,...,958.558472,83.625832,89.970604,77.018280,6.875924,5.345091,NaN,NaN,12.0,1217.916626
2015-01-05 00:00:00+00:00,-0.847833,0.460500,-2.4895,8.4,0.0,5.88,6.881860,314.149841,0.88,NaN,...,968.631409,93.280952,98.554314,85.192207,5.284514,3.080584,NaN,NaN,13.0,765.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28 00:00:00+00:00,4.800083,6.360500,3.2605,0.0,0.0,0.00,10.430724,214.158920,3.15,NaN,...,962.023254,76.966934,85.996208,62.241528,8.925221,7.488657,NaN,NaN,0.0,343.750000
2023-12-29 00:00:00+00:00,7.139667,8.410501,5.1605,0.0,0.0,0.00,9.899495,223.687241,3.39,NaN,...,957.753113,80.429390,90.355545,72.525963,8.493058,6.862215,NaN,NaN,0.0,836.250000
2023-12-30 00:00:00+00:00,5.614666,7.310500,2.8605,0.1,0.1,0.00,8.504117,234.613480,3.24,NaN,...,957.206482,80.916817,92.821678,67.846848,5.735832,3.252691,NaN,NaN,1.0,726.666687


In [10]:
daily_dataframe.index = daily_dataframe.index.tz_convert(None)
ds = daily_dataframe.to_xarray()
ds

<xarray.Dataset> Size: 368kB
Dimensions:                      (date: 3288)
Coordinates:
  * date                         (date) datetime64[ns] 26kB 2015-01-01 ... 20...
Data variables: (12/26)
    temperature_2m_mean          (date) float32 13kB 1.379 2.3 ... 4.027 3.206
    temperature_2m_max           (date) float32 13kB 4.061 4.31 ... 7.31 5.01
    temperature_2m_min           (date) float32 13kB -0.6395 0.4105 ... 1.91
    precipitation_sum            (date) float32 13kB 0.7 0.6 0.0 ... 0.1 0.1 1.9
    rain_sum                     (date) float32 13kB 0.7 0.6 0.0 ... 0.1 0.1 1.9
    snowfall_sum                 (date) float32 13kB 0.0 0.0 0.21 ... 0.0 0.0
    ...                           ...
    wind_speed_10m_mean          (date) float32 13kB 3.429 6.473 ... 5.669 5.405
    wind_speed_10m_min           (date) float32 13kB 2.642 4.465 ... 4.276 3.689
    cape_max                     (date) float32 13kB nan nan nan ... nan nan nan
    cape_min                     (date) float32 13kB nan nan nan ... nan nan nan
    precipitation_hours          (date) float32 13kB 5.0 2.0 0.0 ... 1.0 1.0 9.0
    boundary_layer_height        (date) float32 13kB 259.0 718.3 ... 179.6 nan

In [11]:
ds.to_netcdf("wal_weather.nc")